In [1]:
import boto.ec2
import boto3

In [2]:
%run aws_helper.py

In [3]:
si = SpotInstances()
ctq = ClusterTaskQueue()

In [4]:
sqs = boto3.resource('sqs')  

In [5]:
new_member_queue = sqs.get_queue_by_name(QueueName='cluster_new_member')
task_commitment_queue = sqs.get_queue_by_name(QueueName='cluster_task_commitment')
task_queue = sqs.get_queue_by_name(QueueName='cluster_task_queue')
# instance metadata: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-instance-metadata.html
# ipython setup: http://ipython.org/ipython-doc/dev/notebook/public_server.html#running-a-public-notebook-server

In [7]:
si.request(num_instances=90, subnet='c')

The following script will be run on startup
#!/bin/bash

echo "==========================="
echo "Starting service 'worker-server'."
initctl status worker-server
echo "==========================="



In [8]:
si.print_public_dns_names()

ec2-52-89-29-186.us-west-2.compute.amazonaws.com
ec2-52-89-198-199.us-west-2.compute.amazonaws.com
ec2-52-89-197-16.us-west-2.compute.amazonaws.com
ec2-52-89-198-190.us-west-2.compute.amazonaws.com
ec2-52-11-167-24.us-west-2.compute.amazonaws.com
ec2-52-89-133-158.us-west-2.compute.amazonaws.com
ec2-52-88-216-74.us-west-2.compute.amazonaws.com
ec2-52-89-16-0.us-west-2.compute.amazonaws.com
ec2-52-89-173-146.us-west-2.compute.amazonaws.com
ec2-52-89-201-45.us-west-2.compute.amazonaws.com
ec2-52-89-196-247.us-west-2.compute.amazonaws.com
ec2-52-25-220-1.us-west-2.compute.amazonaws.com
ec2-52-24-119-81.us-west-2.compute.amazonaws.com
ec2-52-89-169-123.us-west-2.compute.amazonaws.com
ec2-52-89-199-105.us-west-2.compute.amazonaws.com
ec2-52-88-193-167.us-west-2.compute.amazonaws.com
ec2-52-89-200-229.us-west-2.compute.amazonaws.com
ec2-52-89-200-246.us-west-2.compute.amazonaws.com
ec2-52-89-201-66.us-west-2.compute.amazonaws.com
ec2-52-89-200-65.us-west-2.compute.amazonaws.com
ec2-52-89-198

In [ ]:
ctq.add_task('hello yejllow 17:27')

In [ ]:
ctq.add_task('git:pull')

In [ ]:
#ctq.add_task('parse:01')
ctq.add_task('parse:test/003:6457:40100')

In [ ]:
ctq.add_task('XXXparse:tmp/01:404075:405000')

In [9]:
run_id = 'prod/04'
total_ids = 91
for i in range(total_ids):
    #ctq.add_task('quit')
    #ctq.add_task('git:pull')
    ctq.add_task('parse:%s:%d:%d' % (run_id, i, total_ids))
    #ctq.add_task('parse:%s' % (run_id))

In [ ]:
ctq.add_task('quit')

In [ ]:
started = []

In [ ]:
for i in range(1):
    msgs = new_member_queue.receive_messages(MessageAttributeNames=['Worker'])
    for msg in msgs:
        worker_dns_name = 'unknown'
        if msg.message_attributes is not None:
            worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
        print('%s: %s' % (worker_dns_name, msg.body))
        msg.delete()
        started.append(worker_dns_name)

In [ ]:
len(started)

In [ ]:
'ec2-52-27-130-165.us-west-2.compute.amazonaws.com' in started

In [ ]:
set(hosts) - set(started)

In [ ]:
msgs = task_commitment_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    msg.delete()

In [ ]:
#Delete 'quit' message after the end
msgs = task_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    #msg.delete()

In [ ]:
new_member_queue.purge()
task_queue.purge()
task_commitment_queue.purge()

In [ ]:
hosts = [
    "ec2-52-89-109-187.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-170.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-184.us-west-2.compute.amazonaws.com",
    "ec2-52-11-209-125.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-202.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-61.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-146.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-179.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-182.us-west-2.compute.amazonaws.com",
    "ec2-52-25-58-77.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-195.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-180.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-110.us-west-2.compute.amazonaws.com",
    "ec2-52-24-26-170.us-west-2.compute.amazonaws.com",
    "ec2-52-89-86-231.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-122.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-167.us-west-2.compute.amazonaws.com",
    "ec2-52-88-244-17.us-west-2.compute.amazonaws.com",
    "ec2-52-89-85-182.us-west-2.compute.amazonaws.com",
    "ec2-52-88-205-245.us-west-2.compute.amazonaws.com",
    "ec2-52-89-98-56.us-west-2.compute.amazonaws.com",
    "ec2-52-27-179-76.us-west-2.compute.amazonaws.com",
    "ec2-52-89-95-232.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-159.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-117.us-west-2.compute.amazonaws.com",
    "ec2-52-89-83-74.us-west-2.compute.amazonaws.com",
    "ec2-52-89-103-159.us-west-2.compute.amazonaws.com",
    "ec2-52-27-130-165.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-177.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-176.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-150.us-west-2.compute.amazonaws.com",
    "ec2-52-11-123-225.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-213.us-west-2.compute.amazonaws.com",
    "ec2-52-89-31-158.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-174.us-west-2.compute.amazonaws.com",
    "ec2-52-89-37-106.us-west-2.compute.amazonaws.com",
    "ec2-52-88-192-88.us-west-2.compute.amazonaws.com",
    "ec2-52-89-102-179.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-196.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-192.us-west-2.compute.amazonaws.com",
    "ec2-52-27-87-249.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-216.us-west-2.compute.amazonaws.com",
    "ec2-52-89-33-170.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-211.us-west-2.compute.amazonaws.com",
    "ec2-52-89-78-197.us-west-2.compute.amazonaws.com",
    "ec2-52-89-108-31.us-west-2.compute.amazonaws.com",
    "ec2-52-89-47-29.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-156.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-121.us-west-2.compute.amazonaws.com",
    "ec2-52-88-66-252.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-140.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-183.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-59.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-208.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-171.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-142.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-206.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-181.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-201.us-west-2.compute.amazonaws.com",
    "ec2-52-89-98-159.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-197.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-234.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-66.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-102.us-west-2.compute.amazonaws.com",
    "ec2-52-89-91-43.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-188.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-135.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-186.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-63.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-194.us-west-2.compute.amazonaws.com",
    "ec2-52-89-109-113.us-west-2.compute.amazonaws.com"
    ]
